# Introduction
* The purpose of this kernel is to convert the collection of 684 job bulletin text files into a single structured .csv file (aka flat file)
* A job bulletin has a hierarchical structure: within the Carpenter bulletin, the job titles "carpenter" and "cabinetmaker" are within the apprenticeship experience requirement; another branch of the tree only requires full time paid experience doing carpenter or cabinetmaker work
* I think that this is the fundamental problem: in a csv, each leaf of the tree should either be on its own line (if it has parents, e.g. a sub-requirement) or in its own column (if it has no parents, e.g. duties or salary). 
* Getting this right will let us feed the hierarchical structure of requirements directly into a graph of promotions
* If this csv format is followed, then every line in the csv should correspond to a path in the promotion graph
* My strategy is to use regular expressions to tokenize the entries in the columns of the structured csv
* To help with this, I've taken some vocabulary from Los Angeles area college course catalogs

In [1]:
## Importing packages

library(tidyverse)
library(tidytext)
library(recipes)
library(quanteda)
#library(hunspell)
library(igraph)
#library(pdftools)
library(rvest)

## Set seed 
set.seed(79)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1.9000     ✔ purrr   0.3.2     
✔ tibble  2.1.1          ✔ dplyr   0.8.1     
✔ tidyr   0.8.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘recipes’

The following object is masked from ‘package:stringr’:

    fixed

The following object is masked from ‘package:stats’:

    step

Package version: 1.4.3
Parallel computing: 2 of 4 threads used.
See https://quanteda.io for tutorials and examples.

Attaching package: ‘quanteda’

The following object is masked from ‘jupyter:irkernel’:

    View

The following object is masked from ‘package:utils’:

    View


Attaching package: ‘igraph’

The following object is masked from ‘package:quanteda’:

    as.igraph

The following objects are masked f

In [2]:
# List files for input
file_list  <- list.files('../input/data-science-for-good-city-of-los-angeles/cityofla//CityofLA//Job Bulletins',
                        full.names = TRUE)
job_titles <- read_csv('../input/data-science-for-good-city-of-los-angeles/cityofla/CityofLA/Additional data/job_titles.csv', col_names = "title") 


Parsed with column specification:
cols(
  title = col_character()
)


In [3]:
# Build keyword dictionaries
# Remove stop words 

job_words <- job_titles$title %>% 
  str_split("_") %>% 
  unlist %>% 
  str_to_lower %>%
  unique %>% 
  str_squish() %>%
  as.character() %>% 
  str_remove_all(pattern = '\\\\"') %>% 
  str_remove_all(pattern="(of|and|code:|only|1968|\\(|\\)|to|2315)") %>% 
  unique

job_words <- job_words[length(job_words > 0)]
 job_words <- job_words[str_length(job_words) > 1]

job_dictionary <- dictionary(list(job_title = job_words))

elac_programs <- read_file("../input/elac-programs/elac_terms.txt")


ucla_programs <-
  read_html("https://catalog.registrar.ucla.edu/ucla-catalog18-19-4.html") %>%
  html_nodes(".main-text") %>%
  html_nodes("li") %>%
  html_nodes("a") %>%
  html_text %>%
  unlist %>%
  str_to_lower %>%
  str_squish() %>%
  unique

ucla_programs <- str_remove_all(ucla_programs,
    regex("[:space:](ba|
    bs|
    ma|
    ms|
    phd|
    llm, jd, sjd|
    dnp|
    md|
    dds|
    denv|
    cphil)$"
  )
) %>% 
  str_replace_all("—", "-") %>% 
  iconv(to = "UTF-8")


major_words <- c(elac_programs, ucla_programs)
majors_dictionary <- dictionary(list(major = major_words))

course_subjects <- c(job_words, major_words)
course_subject_words <- types(tokens(course_subjects))

In [4]:
words_to_digits <- function(text) {
  # It will be a lot easier to make regexes if  I convert all number words to digits first
  text %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bone"), replacement = "1") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\btwo"), replacement = "2") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bthree"), replacement = "3") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bfour"), replacement = "4") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bfive"), replacement = "5") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bsix"), replacement = "6") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bseven"), replacement = "7") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\beight"), replacement = "8") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bnine"), replacement = "9") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\bten"), replacement = "10") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\beleven"), replacement = "11") %>%
    str_replace_all(pattern = regex(ignore_case = TRUE,"\\btwelve"), replacement = "12")
}

In [5]:

# Read titles and class codes from files
# the filenames have more issues 
# e.g. (1).txt, (2).txt, ...,  _two_ spaces after the class code, no class code at all,
# and also filenames with the word REV or Rev or Revised or Updated in them, followed by revision date
# By the way, this might be evidence of the complexity of the task of writing the descriptions (to equitably get
# qualified candidates?)
job_titles <- map(file_list, read_lines, n_max = 1, skip_empty_rows = TRUE) %>% 
    str_squish %>% 
    str_replace_all(string = ., pattern = " ", replacement = "_")

class_codes  <-  map(file_list, read_lines, n_max = 10) %>%
    map(.x = ., .f = grep, pattern = "Class Code:", value = TRUE, ignore.case = TRUE) %>% 
    str_squish 

titles_and_codes  <- paste0(job_titles, class_codes)


# Arrange file contents into a dataframe so that tidytext can use them

file_contents  <- map(file_list, read_file) %>% 
    set_names(x = ., nm = titles_and_codes) %>% 
    unlist

# Perform some ad-hoc spellchecking
file_contents2 <- data.frame(title = unlist(titles_and_codes), text = file_contents, filename = file_list) %>% 
  mutate(text = str_replace_all(text, pattern = "\r\n([[:space:]])*(REQUIREMENT|REQUIREMENTS)([:space:])*\r\n", 
    replacement = "\r\nREQUIREMENTS/MINIMUM QUALIFICATION\r\n") %>%
    str_replace_all(pattern = "/[[:space:]]", replacement = "/") %>%
    str_replace_all(pattern = "[[:space:]]/", replacement = "/") %>%
    str_replace_all(pattern = "MIMINUMUM|MINUMUM", replacement = "MINIMUM") %>% 
    str_replace_all(pattern = "QUALIFICAITON", replacement = "QUALIFICATIONS") %>%  
    str_replace_all(pattern = "(\r\n)*([[:space:]])*REQUIREMENT/MINIMUM QUALIFICATION([[:space:]])*(\r\n)*", replacement = "\r\nREQUIREMENTS/MINIMUM QUALIFICATIONS\r\n") %>% 
    str_replace_all(pattern = "four year college",
        replacement = "four-year college") %>% 
    str_replace_all(pattern = "two year college", 
        replacement = "two-year college") 
    ) %>% 
mutate(text = str_squish(as.character(text)),
  filename = as.character(filename),
  title = as.character(title))

Warning message in stri_replace_all_regex(string, pattern, fix_replacement(replacement), :
“argument is not an atomic vector; coercing”

# Generating a single structured csv

This is 30 or so little problems of tokenizing different aspects of a job bulletin

1. Initial goal is  a prototype dataframe having (I assume) easiest-to-parse fields: FILE\_NAME, JOB\_CLASS\_TITLE and \_NO, OPEN\_DATE, ...
2. For each field, write a function to extract it from the 'text' field of the file_contents2 dataframe above that returns a vector of objects of admissible types from the data dictionary.
3. Map these functions over job descriptions and bind them into a data frame, nesting within job titles
4. Test the data frame for missing fields and allowable values
5. Write to structured\_descriptions.csv
6. Iterate: try to find the best regexes for the job and clean ways to cut out junk

In [6]:
get_job_duties <- function(text) {
  duties_match <-
    str_extract(text, pattern = "(duties[[:space:]])([[:print:]])*(requirements/minimum[[:space:]])") %>%
    str_remove(pattern = "requirements/minimum") %>%
    str_remove(pattern = "duties") %>%
    str_trim() %>%
    str_to_sentence()
  return(duties_match)
}


get_entry_salary <- function(text) {
  # Define a salary regex
  salary_regex <-
    regex(pattern = "(power is[[:space:]])?\\$([[:digit:]]){2,3},([[:digit:]]){3}(\\.[[:digit:]][[:digit:]])?(\\*)?([[:space:]])?(to \\$([[:digit:]]){2,3},([[:digit:]]){3}(\\.[[:digit:]][[:digit:]])?|(\\()?(flat)(\\s)?(-)?(\\s)?(rated)(\\)?)|and)",
      ignore_case = TRUE)
  # Match all salary text
  salary_matches <-
    str_match_all(string = text, pattern = salary_regex)

  # Identify whether general or dwp & clean text
  salary_gen_text <- NA
  salary_dwp_text <- NA
  if (is_empty(salary_matches[[1]])) {
    return(list(salary_gen = salary_gen_text, salary_dwp = salary_dwp_text))

  } else {
    # Also, there are a few jobs without salaries or only having DWP salaries
    if ((sum(!is.na(salary_matches[[1]][, 1])) > 0) &
        (is.na(salary_matches[[1]][1, 2]))) {
      # logic to handle descriptions with no salary or DWP-salary only (airport police specialist, boilermaker and others)
      salary_gen_text <- salary_matches[[1]][1, 1] %>%
        str_replace_all(string = .,
          pattern = "\\$|,",
          replacement = "") %>%
        str_replace(string = .,
          pattern = "[[:space:]]to[[:space:]]",
          replacement = "-") %>%
        str_replace(string = .,
          pattern = "and",
          replacement = "(flat-rated)") %>%
        str_squish()
    }


    if (sum(!is.na(salary_matches[[1]][, 2])) > 0) {
      salary_dwp_text <- salary_matches[[1]] %>%
        as_tibble(.name_repair = "universal") %>%
        filter(!is.na(...2)) %>%
        slice(1) %>%
        str_replace_all(string = .,
          pattern = "\\$|,",
          replacement = "") %>%
        str_replace(string = .,
          pattern = "[[:space:]]to[[:space:]]",
          replacement = "-") %>%
        str_replace(
          string = .,
          pattern = regex(
            ignore_case = TRUE,
            "power is",
            ignore.case = TRUE
          ),
          replacement = ""
        ) %>%
        str_squish() %>%
        `[[`(1)
    }

    # test correct shape of returned object
    return(list(salary_gen = salary_gen_text, salary_dwp = salary_dwp_text))
  }
} # end get salary


get_open_date <- function(text) {
  # Open date from file contents
  open_date <-
    (
      str_trim(str_match(
        string = text,
        pattern = regex(
          "(open date)(:)*([:space:])+([:digit:]){2}(-)([:digit:]){2}(-)([:digit:]){2}",
          ignore_case = TRUE
        )
      )[1, 1]) %>%
        str_match(string = ., pattern = "([:digit:]){2}(-)([:digit:]){2}(-)([:digit:]){2}")
    )[1, 1]

  test_date <-
    try(expr = parse_date(open_date, format = "%m-%d-%y"),
      silent = FALSE)
  ## there are a bunch of these as well: just check first line against filename
  error_flag <-
    ifelse(!is.na(test_date), "No Error", "Date Format Error")
  return(list(open_date = open_date, error_flag = error_flag))
}

# Get all requirement sets and subsets
get_requirement_sets_and_details <- function(text) {
  ## Function to return three pieces
  ## Requirement sets and subsets
  ## Process notes
  ## Other text (ex: selective certifications for systems analyst)
  req_s_and_d_regex <-
    regex(ignore_case = TRUE, pattern = "(requirements/minimum)([[:print:]])*(where to apply)")

  process_notes_regex <-
    regex(ignore_case = TRUE, pattern = "(process )? (notes)([[:print:]])*(where to apply)")

  req_matches <-
    str_match_all(string = text, pattern = req_s_and_d_regex)
  process_matches <-
    str_match_all(string = text, pattern = process_notes_regex)

  if ((!is_empty(process_matches[[1]])) &
      (!is_empty(req_matches[[1]]))) {
    req_matches[[1]][1, 1] <-
      str_split(req_matches[[1]][1, 1], pattern = "(process )? (notes)")[[1]][1]
  }

  reqs_text <- NA
  process_text <- NA

  if (is_empty(req_matches[[1]])) {
    return(list(reqs_text = reqs_text, process_text = process_text))

  } else {
    reqs_text <- req_matches[[1]][1, 1] %>%
      str_remove_all(regex("where to apply", ignore_case = TRUE)) %>%
      str_remove_all(regex("requirements/minimum qualification.", ignore_case = TRUE)) %>%
      str_remove_all(regex("process", ignore_case = TRUE)) %>%
      str_squish()
  }


  if (is_empty(process_matches[[1]])) {
    return(list(reqs_text = reqs_text, process_text = process_text))
  } else {
    process_text <- process_matches[[1]][1, 1] %>%
      str_remove_all(regex("where to apply", ignore_case = TRUE)) %>%
      str_remove_all(regex("process notes", ignore_case = TRUE)) %>%
      str_squish()
  }

  return(list(reqs_text = reqs_text, process_text = process_text))
} # end extract all requirements text

# Get requirement sets and subsets based on enumerations/itemize line starts and conjunctions
# This is actually a tree structure, but knowing that doesn't help
# anything
get_requirement_set <- function(text) {
  split_req <- str_split(text, pattern = "; or [[:digit:]]")
  reqset <- data.frame(split_req) %>% rownames_to_column()
  colnames(reqset) <- c("REQUIREMENT_SET_ID", "REQUIREMENTS")
  reqset <-
    mutate(reqset, REQUIREMENTS = as.character(REQUIREMENTS))
  return(reqset)
}

get_requirement_subset <- function(text) {
  split_sreq  <- str_split(text, pattern = "; or [[:alpha:]]\\.")
  sreqset  <- data.frame(split_sreq) %>% rownames_to_column()
  colnames(sreqset) <- c("REQUIREMENT_SUBSET_ID", "REQUIREMENTS")
  sreqset  <-
    mutate(
      sreqset,
      REQUIREMENT_SUBSET_ID = as.character(REQUIREMENT_SUBSET_ID),
      REQUIREMENTS = as.character(REQUIREMENTS)
    ) %>%
    mutate(
      REQUIREMENTS = str_remove_all(REQUIREMENTS,
        pattern = "^([:digit:])?([:punct:])") %>%
        str_squish()
    )
  return(sreqset)
}

# Apply these functions to all requirement sets and subsets

## Parse Education Requirements

get_school_type <- function(text) {
  school_type_dictionary <-
    data.frame(
      token = c(
        "4-year college",
        "university",
        "2-year college",
        "community college",
        "trade school",
        "technical school",
        "high school",
        "g.e.d.",
        "(police|lafd leadership) academy",
        "art school",
        "training program",
        "chspe",
        "military school",
        "lineman college",
        "lineman's college"
      ),
      type = c(
        "college or university",
        "college or university",
        "2-year college",
        "2-year college",
        "technical training",
        "technical training",
        "secondary school",
        "secondary school",
        "technical training",
        "technical training",
        "technical training",
        "secondary school",
        "technical training",
        "technical training",
        "technical training"
      )
    ) %>%
    mutate_all(as.character)

  school_types <-
    regex(
      ignore_case = TRUE,
      pattern = paste(school_type_dictionary$token, collapse = "|")
    )

  school_matches <- str_match_all(text, pattern = school_types)
  if (!is_empty(school_matches[[1]][, 1])) {
    return(
      school_type_dictionary %>%
        filter(token %in% school_matches[[1]][, 1]) %>%
        pull(type) %>%
        unique %>%
        paste(collapse = " or ")
    )
  } else {
    return(NA)
  }
} # end get school type

get_education_major <- function(text) {
  major_regex <-
    regex(ignore_case = TRUE,
      "((degree|major) in)([:print:])*(and|from|\\.)")
  major_matches <- str_match_all(text, major_regex)
  course_words_rx <-
    regex(ignore_case = TRUE, paste(course_subject_words, collapse = "|"))

  major_words   <-
    str_extract_all(string = major_matches[[1]][, 1], course_words_rx) %>% unlist %>% paste(collapse = " ")

  if (!is_empty(major_matches[[1]][, 1])) {
    return(
      list(
        major_matches = str_split(major_matches[[1]][1, 1], major_matches[[1]][1, 2])[[1]][2] %>%
          str_split(pattern = major_matches[[1]][1, 5]) %>%
          `[[`(1) %>%
          `[`(1) %>%
          str_squish(),
        major_words = major_words
      )
    )
  } else {
    return(NA)
  }
} # end get major

get_course_length <- function(text) {
  course_len_regex <-
    regex(ignore_case = TRUE,
      "[:digit:]{1,3} (quarter|semester|weeks) (unit)?(s)?")
  course_len_match <-
    str_match_all(string = text, pattern = course_len_regex)

  if (!is_empty(course_len_match[[1]][, 1])) {
    return(course_len_match[[1]][, 1] %>%
        paste(collapse = ",") %>%
        str_squish())
  } else {
    return(NA)
  }
}

get_course_subject_and_count <- function(text) {
  course_regex <-
    regex(
      ignore_case = TRUE,
      "(successful )?(completion (of|for)[:print:]*)?([:digit:]{0,3}|a)?([[:print:]-\\.]*)? (class|course|units|program)([[:print:]-\\.]*)(from|given|\\.)"
    )
  course_matches <- str_match_all(text, course_regex)
  course_words_rx <-
    regex(ignore_case = TRUE, paste(course_subject_words, collapse = "|"))
  course_words   <-
    str_extract_all(string = course_matches[[1]][, 1], course_words_rx) %>% unlist %>% paste(collapse = " ")

  if (!is_empty(course_matches[[1]][, 8])) {
    return(
      list(
        subject = course_matches[[1]][, 8] %>%
          str_squish(),
        count = course_matches[[1]][, 4],
        course_words = course_words
      )
    )
  } else {
    return(NA)
  }
}


## Parse Experience Requirements
get_experience_length <- function(text) {
  experience_regex <-
    regex(ignore_case = TRUE, pattern = "([[:digit:]])+ (years|months) of (full|part)(-)?time (paid|volunteer) experience ([[:print:]])*\\.")
  experience_matches <-
    str_match_all(text, pattern = experience_regex)
  return(experience_matches)
}

get_exp_job_class_title <- function(text) {
  job_title_sentence <-
    regex(
      ignore_case = TRUE,
      "(experience[:print:]*)((as a)|(at th(e|at) level))([:print:]*)(;|\\.)"
    )
  job_words_rx <-
    regex(ignore_case = TRUE, paste(job_words, collapse = "|"))
  job_function_regex <-
    regex(ignore_case = TRUE, "([:print:]*ing)([:print:]*)(;|\\.)")

  job_sentences <- str_match_all(text, job_title_sentence)
  exp_job_words <-
    str_extract_all(string = job_sentences[[1]][, 1], job_words_rx) %>% unlist %>% paste(collapse = " ")
  job_functions <-
    str_extract_all(string = job_sentences[[1]][, 1], pattern = job_function_regex)  %>% unlist %>% paste(collapse = " ")

  ## TODO: add get alternative responsibilities

  if (!is_empty(job_sentences[[1]][, 7])) {
    return(

        exp_job_title = job_sentences[[1]][,7]

    )
  } else {
    return(NA)
  }
}




# Drivers' licences
get_drivers_license_req <- function(text) {
  dl_regex <-
    regex(ignore_case = TRUE, pattern = "(valid california class)([:print:]*) (driver(')?(s)? licen(s|c)e)")
  dl_matches <- str_match_all(text, pattern = dl_regex)


  if (!is_empty(dl_matches[[1]][, 3])) {
    return(list(
      required_dl_class = dl_matches[[1]][1, 3] %>% str_remove_all(regex("class", ignore_case = TRUE)) %>% str_squish()
    ))
  } else{
    return(NA)
  }

}

## Parse examination requirement (not contained in requirements)
get_exam_type <- function(text) {
  exam_regex <-
    regex(ignore_case = TRUE,
      "(this examination is to be given)([:print:]*)(the city of los angeles)")
  str_match(text, exam_regex)[[3]] %>%
    str_squish() %>%
    str_extract_all(pattern = "(open (competitive)?)|((interdepartmental)? promotional)") %>%
    unlist %>%
    paste(collapse = ", ") %>%
    return()
}


get_exp_job_class_function <- function(text){

  experience_fn_regex <-
    regex(ignore_case = TRUE, pattern = "(experience in)([[:print:]])*(\\.|;)")
  experience_matches <-
    str_match(text, pattern = experience_fn_regex)
   return(experience_matches)

}


# Generating a structured csv
- Using the ideal of a job bulletin as a tree and one line (leaves of depth >1) or one column (leaves of depth 1) per leaf on the tree, I nest within job bulletin titles and generate a few rows for each one; then I bind rows together.
- I wrote one function for each set of related columns, so I generate one dataframe for each set of such columns
- After that, I left-join all of these little dataframes into one big one which I plan to output to csv.
- There's still a lot of garbage in the big dataframe, so I need to define tests on it 

In [7]:
open_dates <- file_contents2 %>%
  group_by(title) %>%
  do(.data = .,
    data.frame(get_open_date(.$text)) %>%
      mutate_if(is.factor, as.character)) %>%
  ungroup

suppressMessages(
  salaries <- file_contents2 %>%
    group_by(title) %>%
    do(
      .data = .,
      data.frame(get_entry_salary(.$text)) %>%
        mutate_if(is.factor, as.character)
    ) %>%
    ungroup
)

duties <- file_contents2 %>%
  group_by(title) %>%
  do(.data = .,
    data.frame(get_job_duties(.$text)) %>%
      mutate_if(is.factor, as.character)) %>%
  ungroup
requirements_and_details_to_parse <-
  file_contents2 %>%
  group_by(title) %>%
  do(
    .data = .,
    data.frame(get_requirement_sets_and_details(.$text)) %>% mutate_if(is.factor, as.character)
  ) %>%
  ungroup


requirement_sets <- requirements_and_details_to_parse %>%
  group_by(title) %>%
  do(.data = .,
    data.frame(get_requirement_set(.$reqs_text)) %>% mutate_if(is.factor, as.character)) %>%
  ungroup()


requirement_subsets <- requirement_sets %>%
  group_by(title, REQUIREMENT_SET_ID) %>%
  do(.data = .,
    data.frame(get_requirement_subset(.$REQUIREMENTS)) %>% mutate_if(is.factor, as.character)) %>%
  ungroup %>%
  mutate(
    REQUIREMENT_SUBSET_ID = recode(
      REQUIREMENT_SUBSET_ID,
      `1` = "A",
      `2` = "B",
      `3` = "C",
      `4` = "D",
      `5` = "E",
      `6` = "F",
      `7` = "G",
      `8` = "H",
      `9` = "I",
      `10` = "J",
      `11` = "K",
      `12` = "L",
      `13` = "M",
      `14` = "N",
      `15` = "O",
      `16` = "P",
      `17` = "Q",
      `18` = "R",
      `19` = "S",
      `20` = "T",
      `21` = "U",
      `22` = "V",
      `23` = "W",
      `24` = "X",
      `25` = "Y",
      `26` = "Z",
      `27` = "AA",
      `28` = "AB",
      `29` = "AC",
      `30` = "AD",
      `31` = "AE",
      `32` = "AF",
      `33` = "AH",
      `34` = "AI",
      `35` = "AJ",
      `36` = "AK"
    ),
    REQUIREMENTS = words_to_digits(REQUIREMENTS)
  )

process_notes <- requirements_and_details_to_parse %>%
  group_by(title) %>%
  do(
    .data = .,
    data.frame(get_requirement_set(.$process_text)) %>% mutate_if(is.factor, as.character) %>%
      rename(process_notes = 'REQUIREMENTS')
  ) %>%
  ungroup

experience_subset <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(
    .data = .,
    data.frame(get_experience_length(.$REQUIREMENTS))  %>% mutate_if(is.factor, as.character) %>%
      rename(
        experience = X1,
        length = X2,
        unit = X3,
        full_time_part_time = X4,
        drop1 = X5,
        paid_volunteer = X6,
        drop2 = X7
      )
  ) %>%
  select(-starts_with("drop")) %>%
  ungroup

experience_process <- process_notes %>%
  group_by(title,  REQUIREMENT_SET_ID) %>%
  do(
    .data = .,
    data.frame(get_experience_length(.$process_notes))  %>% mutate_if(is.factor, as.character) %>%
      rename(
        experience = X1,
        length = X2,
        unit = X3,
        full_time_part_time = X4,
        drop1 = X5,
        paid_volunteer = X6,
        drop2 = X7
      )
  ) %>%
  select(-starts_with("drop")) %>%
  ungroup

dl_subset <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(
    .data = .,
    data.frame(get_drivers_license_req(.$REQUIREMENTS))  %>% mutate_if(is.factor, as.character)
  ) %>%
  ungroup

dl_process <- process_notes %>%
  group_by(title,  REQUIREMENT_SET_ID) %>%
  do(
    .data = .,
    data.frame(get_drivers_license_req(.$process_notes))  %>% mutate_if(is.factor, as.character)

  ) %>%
  ungroup

school_type <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(.data = .,
    data.frame(get_school_type(.$REQUIREMENTS))  %>% mutate_if(is.factor, as.character)) %>%
  ungroup

major <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(.data = .,
    data.frame(get_education_major(.$REQUIREMENTS))  %>% mutate_if(is.factor, as.character)) %>%
  ungroup

course_lengths <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(
    .data = .,
    data.frame(course_length = get_course_length(.$REQUIREMENTS))  %>% mutate_if(is.factor, as.character)
  ) %>%
  ungroup


course_subject_and_count <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(
    .data = .,
    data.frame(get_course_subject_and_count(.$REQUIREMENTS))  %>% mutate_if(is.factor, as.character)
  ) %>%
  ungroup

job_title_subset <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(
    .data = .,
    data.frame(get_exp_job_class_title(.$REQUIREMENTS)) %>%
      mutate_if(is.factor, as.character)
  ) %>%
  ungroup

job_class_function <- requirement_subsets %>%
  group_by(title, REQUIREMENT_SET_ID, REQUIREMENT_SUBSET_ID) %>%
  do(
    .data = .,
    exp_job_class_function = data.frame(get_exp_job_class_function(.$REQUIREMENTS)) %>%
      mutate_if(is.factor, as.character)
  ) %>%
  ungroup

examinations <- file_contents2 %>%
  group_by(title) %>%
  mutate(EXAM_TYPE = get_exam_type(text)) %>%
  ungroup


nonfunctional_prototype_csv <-
  left_join(requirement_subsets, experience_subset) %>%
  left_join(open_dates) %>%
  left_join(duties) %>%
  left_join(salaries) %>%
  left_join(dl_subset) %>%
  left_join(dl_process) %>%
  # left_join(lic_subset) %>%
  # left_join(lic_process) %>% delete these functions!
  left_join(school_type) %>%
  ## left_join(major) %>%  delete this function
  # left_join(course_subject_and_count) %>%
  left_join(course_lengths) %>%
  left_join(job_title_subset) %>%
  left_join(job_class_function) %>%
  separate(title,
    into = c('job_title', 'class_code'),
    sep = "Class Code: ") %>%
  mutate(
    education_years = str_replace_all(
      get_school_type...REQUIREMENTS.,
      "postgraduate college or university",
      "6+"
    ) %>%
      str_replace_all("college or university", "4") %>%
      str_replace_all("2-year college", "2") %>%
      str_remove_all(regex(ignore_case = TRUE, "(or )?technical training")) %>%
      str_remove_all(regex(ignore_case = TRUE, "(or )?secondary school")) %>%
      str_squish()
  ) %>%
  unite(col = "experience_length", length, unit, sep = " ")


# build a csv by keyword lookup in context ----
experience_job_class_title <-
  kwic(nonfunctional_prototype_csv$REQUIREMENTS,
    pattern = job_dictionary) %>%
  filter(!grepl("license", x = pre),
    !grepl("license", x = post),
    !grepl("senior", x = pre)) %>%
  select(docname, keyword) %>%
  mutate(doc_row = as.integer(str_remove_all(docname, "text"))) %>%
  group_by(docname, doc_row) %>%
  summarize(class_titles = paste(unique(as.character(keyword)), collapse = ", "))

nonfunctional_prototype_csv$exp_job_class_title <- NA
nonfunctional_prototype_csv$exp_job_class_title[experience_job_class_title$doc_row] <-
  experience_job_class_title$class_titles

ed_major <- kwic(nonfunctional_prototype_csv$REQUIREMENTS,
  pattern = majors_dictionary,
  window = 36) %>%
  filter(grepl("degree|graduation", x = pre)) %>%
  select(docname, keyword) %>%
  mutate(doc_row = as.integer(str_remove_all(docname, "text"))) %>%
  group_by(docname, doc_row) %>%
  summarize(class_titles = paste(unique(as.character(keyword)), collapse = ", "))

nonfunctional_prototype_csv$education_major <- NA
nonfunctional_prototype_csv$education_major[ed_major$doc_row] <-
  ed_major$class_titles

addtl_license <-
  kwic(
    nonfunctional_prototype_csv$REQUIREMENTS,
    pattern = c(
      "license",
      "licensed",
      "certificate",
      "certified",
      "cert",
      "certs",
      "certification"
    ),
    window = 24
  ) %>%
  filter(!grepl("driver", x = pre)) %>%
  select(docname, pre, keyword, post) %>%
  mutate(
    doc_row = as.integer(str_remove_all(docname, "text")),
    pre = str_extract(
      pre,
      pattern = regex(
        ignore_case = TRUE,
        "(must|(;[:space:])?and|[:upper:]|[:punct:])([[:print:]]*)$"
      )
    ),
    post = str_extract(
      post,
      pattern = regex(
        ignore_case = TRUE,
        "^[[:print:]]*(\\band\\b|[[:punct:]]|required)"
      )
    )
  ) %>%
  unite(pre,
    keyword,
    post,
    col = "add_lic",
    sep = " ",
    remove = FALSE) %>%
  group_by(docname, doc_row) %>%
  summarize(
    additional_license = paste(unique(as.character(add_lic)), collapse = " "),
    adpost = paste(unique(as.character(post)), collapse = " ")
  ) %>%
  mutate(additional_license = paste(types(
    tokens_remove(
    tokens(additional_license),
      stopwords())), collapse = " ")) %>%
  ungroup
#
nonfunctional_prototype_csv$addtl_lic <- NA
nonfunctional_prototype_csv$addtl_lic[addtl_license$doc_row] <-
  addtl_license$additional_license
#
# nonfunctional_prototype_csv$addl_post <- NA
# nonfunctional_prototype_csv$addl_post[addtl_license$doc_row] <- addtl_license$adpost

addtl_license_process_notes <-
  kwic(
    process_notes$process_notes,
    pattern = c(
      "license",
      "licensed",
      "certificate",
      "certified",
      "cert",
      "certs",
      "certification"
    ),
    window = 24
  ) %>%
  filter(!grepl("driver", x = pre)) %>%
  select(docname, pre, keyword, post) %>%
  mutate(
    doc_row = as.integer(str_remove_all(docname, "text")),
    pre = str_extract(
      pre,
      pattern = regex(
        ignore_case = TRUE,
        "(must|(;[:space:])?and|[:upper:]|[:punct:])([[:print:]]*)$"
      )
    ),
    post = str_extract(
      post,
      pattern = regex(
        ignore_case = TRUE,
        "^[[:print:]]*(\\band\\b|[[:punct:]]|required)"
      )
    )
  ) %>%
  unite(pre,
    keyword,
    post,
    col = "add_lic",
    sep = " ",
    remove = FALSE) %>%
  group_by(docname, doc_row) %>%
  summarize(
    additional_license = paste(unique(as.character(add_lic)), collapse = " "),
    adpost = paste(unique(as.character(post)), collapse = " ")
  ) %>%
  mutate(additional_license = paste(
    types(
      tokens_remove(
        tokens(additional_license),
        stopwords())), collapse = " ")) %>%
  ungroup



Joining, by = c("title", "REQUIREMENT_SET_ID", "REQUIREMENT_SUBSET_ID")
Joining, by = "title"
Joining, by = "title"
Joining, by = "title"
Joining, by = c("title", "REQUIREMENT_SET_ID", "REQUIREMENT_SUBSET_ID")
Joining, by = c("title", "REQUIREMENT_SET_ID", "required_dl_class")
Joining, by = c("title", "REQUIREMENT_SET_ID", "REQUIREMENT_SUBSET_ID")
Joining, by = c("title", "REQUIREMENT_SET_ID", "REQUIREMENT_SUBSET_ID")
Joining, by = c("title", "REQUIREMENT_SET_ID", "REQUIREMENT_SUBSET_ID")
Joining, by = c("title", "REQUIREMENT_SET_ID", "REQUIREMENT_SUBSET_ID")
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 3 rows [322, 634, 635].”

# Testing and output to CSV
Important facts: 
1. Not every job has every kind of requirement
2. Not every requirement is formatted in the same way

Tests
1. Does every job listing have at least 1 experience or education requirement?
2. Do all jobs with 'driver' in the title require licenses?
3. Are college majors represented as words/short phrases?
4. Are course subjects represented as words/short phrases?
5. Does every course requirement have a matching course count?
6. Does every education requirement have a matching education length? Must it?
7. Are required job ranks (e.g. carpenter's _aide_) being captured?
8. Are any fields longer than 140 characters?
9. Does every line represent a subrequirement that is preceded by an _or_ in the bulletin? Subrequirements conjoined with _and_ should be on the same line. 

In [8]:
# Testing code
## Not formally testing yet

# Any jobs missing ed & exp requirements?
  # Unlicensed "drivers"?
# Longest descriptions
# Does every course requirement have a matching course count?
# Does every education requirement have a matching education length? Must it?
# Are required job ranks (e.g. carpenter's aide) being captured?
# Does every line represent a subrequirement that is preceded by an or in the bulletin? Subrequirements conjoined with and should be on the same line.


export_csv <- nonfunctional_prototype_csv %>%
  select(-contains("..."), -error_flag, -exp_job_class_function)

write_csv(export_csv, path = "ds4gLA_structured_bulletins.csv")


In [9]:
# Generate data dictionary (use clean_names() first!)
given_data_dictionary  <- read_csv("../input/data-science-for-good-city-of-los-angeles/cityofla/CityofLA/Additional data/kaggle_data_dictionary.csv")
output_data_dictionary   <- data.frame(field_name = NULL, # colnames(nonfunctional_prototype_csv),
                                annotation_letter = NULL,
                                description = NULL,
                                data_type = NULL,
                                allowable_values = NULL,
                                accepts_null = NULL,
                                additional_notes = NULL
                               )


## Add custom columns and descriptions

Parsed with column specification:
cols(
  `Field Name` = col_character(),
  `Annotation Letter` = col_character(),
  Description = col_character(),
  `Data Type` = col_character(),
  `Allowable Values` = col_character(),
  `Accepts Null Values?` = col_character(),
  `Additional Notes` = col_character()
)
